In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [154]:
def check_df(df) :
    print(df.head())
    for i, col_name in enumerate(df) :
        print(i, col_name, df[col_name].dtype)
        if df[col_name].hasnans :
            print(df[col_name].isna().value_counts())
        print('')
        if len(df[col_name].unique()) < 10 :
            print(df[col_name].value_counts())
        print('')

In [155]:
pd_data = pd.read_csv('전국평생학습강좌표준데이터.csv', encoding='cp949')

# print(pd_data.head())
check_df(pd_data)

           강좌명  강사명      교육시작일자      교육종료일자 교육시작시각 교육종료시각  \
0    생활요리 (주간)  차정미  2021-09-08  2021-12-22  10:00  13:00   
1  헤어실무디자인(야간)  김종순  2021-09-06  2021-12-20  18:30  21:00   
2  헤어실무디자인(주간)  이경희  2021-09-07  2021-12-21  13:30  16:30   
3       옷만들기중급  신선희  2021-09-08  2021-12-24  13:00  15:00   
4       옷만들기초급  신선희  2021-09-08  2021-12-24  15:30  17:30   

                                 강좌내용        교육대상구분 교육방법구분 운영요일  ...  \
0                            생활요리 배우기  만19세 이상 안성시민     혼합    목  ...   
1  현장에서 활용할 수 있는 응용실무교육(미용자격증소지자만 가능)  만19세 이상 안성시민     혼합  화+목  ...   
2  현장에서 활용할 수 있는 응용실무교육(미용자격증소지자만 가능)  만19세 이상 안성시민     혼합  화+목  ...   
3                   상의(남방, 블라우스, 티셔츠)  만19세 이상 안성시민     혼합  화+목  ...   
4                    남성복,여성하의(스커트,바지)  만19세 이상 안성시민     혼합  화+목  ...   

       접수종료일자  접수방법구분  선정방법구분                                 홈페이지주소  \
0  2021-08-25     인터넷     추첨제  https://www.anseong.go.kr/edu/main.do   
1  2021-08-25     인터넷     추첨제  https://www.anseong.go.kr

In [156]:
# pd_data['운영기관명'].unique()
# pd_data['제공기관명'].unique()
pd_data['교육장도로명주소'].unique()

array(['경기도 안성시 강변남로 36', '서울특별시 강서구 공항대로 615', '강원도 인제군 인제읍 하추로 187',
       ..., '강원도 평창군 평창읍 평창중앙로 67', '강원도 평창군 대화면 선애길 90',
       '강원도 평창군 대화면 해뜰마을길 51'], dtype=object)

In [157]:
# 주소 형태 전처리 1 (시도명만 가져오기)
def area_from_address(full_address_str) :
    # case 1: [12345] 지역명 형태 처리
    if '[' in full_address_str :
        i = full_address_str.find(']') + 1  # [12345]가 끝나는 지점 + 1 = 지역명 시작점
        full_address_str = full_address_str[i:]
    
    # case 2 : 띄어쓰기 대신 ?를 사용해 구분한 형태 처리
    if '?' in full_address_str :
        full_address_str = full_address_str.split('?')[0]
    
    return full_address_str.split()[0]

In [158]:
# 주소 형태 전처리 2 (시도명 길이 맞추기)
def rewrite(area_name) :
    # 서울 -> 서울특별시, 부산 -> 부산광역시, 전북 -> 전라북도 ...
    short_name = ['경남', '경북', '전남', '전북', '부산', '광주', '서울', '인천']
    long_name = ['경상남도', '경상북도', '전라남도', '전라북도', '부산광역시', '광주광역시', '서울특별시', '인천광역시']

    if area_name in short_name :
        i = short_name.index(area_name)
        return long_name[i]
    else :
        return area_name

# 주소 형태 전처리 2-1 (시도명이 아닌 구군으로 적은 장소 일부)
def rewrite_2(area_name) :
    if area_name == '마산합포구' :
        return '경상남도'
    elif area_name == '고흥군' :
        return '전라남도'
    elif area_name == '당진시' :
        return '충청남도'
    else :
        return area_name

In [159]:
area_names = pd_data['교육장도로명주소'].apply(area_from_address)
area_names = area_names.apply(rewrite)
area_names = area_names.apply(rewrite_2)

# print(area_names.unique())


# 나머지 처리되지 않은 값은 예외 처리
cnt = area_names.value_counts()
# print(cnt)

ignore_names = list(cnt['.':].index)
# print(ignore_names)

for i, n in enumerate(area_names) :
    if n in ignore_names :
        area_names[i] = '예외'

print(area_names.unique())
pd_data['area_names'] = area_names

['경기도' '서울특별시' '강원도' '경상남도' '예외' '경상북도' '충청북도' '부산광역시' '대구광역시' '울산광역시'
 '광주광역시' '충청남도' '대전광역시' '전라북도' '전라남도' '인천광역시' '세종특별자치시' '제주특별자치도']


In [160]:
# 테스트 케이스

test_addr = '경기도 안성시 강변남로 6'
test_addr1 = '전라남도?진도군?진도읍?철마길12'
test_addr2 = '[46762] 부산광역시'

print(test_addr.split(' ')[0])
print(test_addr1.split('?')[0].split()[0])
print(test_addr2[test_addr2.find(']')+1:].split()[0])
# print(test_addr2[test_addr2.find(']')+1:].strip())


경기도
전라남도
부산광역시


In [161]:
# 테스트 케이스

test_area = '마산합포구 ~~'
test_area_2 = '고흥군 ~~'

print(rewrite_2(rewrite(area_from_address(test_area))))
print(rewrite_2(rewrite(area_from_address(test_area_2))))

경상남도
전라남도


In [162]:
# 테스트 케이스

short_name = ['경남', '경북', '전남', '전북', '부산', '광주', '서울']
long_name = ['경상남도', '경상북도', '전라남도', '전라북도', '부산광역시', '광주광역시', '서울특별시']

str1 = '전남'
i = short_name.index(str1)
long_name[i]

'전라남도'

In [163]:
# print(pd_data.columns)
print(pd_data['area_names'].value_counts())

경기도        8149
경상남도       2767
광주광역시      2214
서울특별시      1968
전라북도       1631
대구광역시      1627
인천광역시      1395
경상북도       1347
충청남도       1048
강원도         916
전라남도        887
부산광역시       844
대전광역시       534
제주특별자치도     467
충청북도        446
예외          428
울산광역시       296
세종특별자치시     146
Name: area_names, dtype: int64


In [164]:
delivery_types = pd_data['교육방법구분']
# delivery_types.unique()

# 교육방법구분 전처리
# +로 되어 있는 방법들을 혼합으로 묶음 처리
def re_type(type_name) :
    if '+' in type_name :
        return '혼합'
    else :
        return type_name

In [165]:
delivery_types = delivery_types.apply(re_type)

# 다른 방법 : 람다식 활용
delivery_types = delivery_types.apply(lambda type_name : '혼합' if '+' in type_name else type_name)

print(delivery_types.unique())
print(delivery_types.value_counts())

# pd_data['delivery_types'] = delivery_types
pd_data.insert(28, 'delivery_types', delivery_types)

['혼합' '오프라인' '온라인' '방문교육']
오프라인    21616
온라인      3852
혼합       1297
방문교육      345
Name: 교육방법구분, dtype: int64


In [166]:
# area_names, delivery_types가 정상 반영되어 있는지 확인
print(pd_data.iloc[:, -5:].head())

      데이터기준일자   제공기관코드    제공기관명 area_names delivery_types
0  2021-09-06  4080000  경기도 안성시        경기도             혼합
1  2021-09-06  4080000  경기도 안성시        경기도             혼합
2  2021-09-06  4080000  경기도 안성시        경기도             혼합
3  2021-09-06  4080000  경기도 안성시        경기도             혼합
4  2021-09-06  4080000  경기도 안성시        경기도             혼합


In [167]:
sub_df = pd_data.iloc[:,-2:]
# print(sub_df)

In [168]:
# 테스트 케이스

filter1 = sub_df['area_names'] == '경기도'
area_df = sub_df[filter1]
area_df['delivery_types'].value_counts()

오프라인    6509
온라인      792
혼합       652
방문교육     196
Name: delivery_types, dtype: int64

In [169]:
for area_name in sub_df['area_names'].unique() :
    print(area_name)
    filter1 = sub_df['area_names'] == area_name
    area_df = sub_df[filter1]
    print(area_df['delivery_types'].value_counts())
    print('')

경기도
오프라인    6509
온라인      792
혼합       652
방문교육     196
Name: delivery_types, dtype: int64

서울특별시
오프라인    1378
온라인      524
혼합        66
Name: delivery_types, dtype: int64

강원도
오프라인    738
온라인     170
혼합        8
Name: delivery_types, dtype: int64

경상남도
오프라인    2695
온라인       72
Name: delivery_types, dtype: int64

예외
오프라인    370
온라인      57
혼합        1
Name: delivery_types, dtype: int64

경상북도
오프라인    1036
온라인      309
혼합         1
방문교육       1
Name: delivery_types, dtype: int64

충청북도
오프라인    298
혼합      130
온라인      18
Name: delivery_types, dtype: int64

부산광역시
오프라인    724
온라인      57
방문교육     45
혼합       18
Name: delivery_types, dtype: int64

대구광역시
오프라인    1591
온라인       31
혼합         5
Name: delivery_types, dtype: int64

울산광역시
오프라인    267
온라인      28
방문교육      1
Name: delivery_types, dtype: int64

광주광역시
오프라인    1805
온라인      315
혼합        94
Name: delivery_types, dtype: int64

충청남도
오프라인    971
온라인      35
방문교육     33
혼합        9
Name: delivery_types, dtype: int64

대전광역시
오프라인    336
혼합

In [170]:
# groupby 활용
sub_df = pd_data[['delivery_types','area_names', '수강료']]
# print(sub_df.head())

g1 = sub_df.groupby(['area_names', 'delivery_types'])
# g1.count()
# g1.mean()
df10 = g1.agg([np.mean, np.max])
df10

수강료         
                                    mean     amax
area_names delivery_types                        
강원도        오프라인             13726.626016   800000
           온라인              31235.294118    80000
           혼합               11250.000000    50000
경기도        방문교육             67755.102041   100000
           오프라인             31995.324935   400000
           온라인              16100.505051   160000
           혼합               34959.739264   112500
경상남도       오프라인             47541.372913   500000
           온라인               1111.111111    40000
경상북도       방문교육                 0.000000        0
           오프라인             19505.501931   305000
           온라인                258.899676    40000
           혼합                   0.000000        0
광주광역시      오프라인             23493.074792   210000
           온라인               7396.825397    80000
           혼합                 425.531915    40000
대구광역시      오프라인             87429.604023  2000000
           온라인              17612.903226   400000
           혼합              180000.000000   500000
대전광역시      오프라인             32589.285714    50000
           온라인              13333.333333    40000
           혼합               32461.538462    40000
부산광역시      방문교육                 0.000000        0
           오프라인               759.792818    25000
           온라인               1929.842105    25000
           혼합                   0.000000        0
서울특별시      오프라인             19178.519594   180000
           온라인               7232.442748    96000
           혼합                8681.818182    81000
세종특별자치시    오프라인             32417.582418    50000
           온라인              13888.888889    50000
           혼합                   0.000000        0
예외         오프라인             45981.081081   440000
           온라인               3508.771930    25000
           혼합                   0.000000        0
울산광역시      방문교육                 0.000000        0
           오프라인             18535.580524   135000
           온라인               1285.714286    24000
인천광역시      오프라인             20287.719298   400000
           온라인               9930.041152   160000
           혼합               35000.000000    80000
전라남도       방문교육                 0.000000        0
           오프라인              6374.399249   100000
           온라인              24000.000000    45000
           혼합                   0.000000        0
전라북도       방문교육                 0.000000        0
           오프라인              5847.866419   100000
           온라인                  0.000000        0
           혼합               54444.444444    70000
제주특별자치도    오프라인             23161.094225    40000
           온라인              27575.757576    40000
           혼합                   0.000000        0
충청남도       방문교육                 0.000000        0
           오프라인              9841.661174   220000
           온라인                  0.000000        0
           혼합                1666.666667     5000
충청북도       오프라인             18775.167785    40000
           온라인               6666.666667    40000
           혼합               27153.846154    50000

In [173]:
# Multi index

# print(df10.index)     # row 인덱스가 멀티 인덱스인 것을 확인
# print(df10.columns)     # column 인덱스가 멀티 인덱스인 것을 확인

# df10[('수강료', 'mean')]
# df10[('수강료', 'amax')]

print(df10.loc[('강원도', '온라인')])


수강료  mean    31235.294118
     amax    80000.000000
Name: (강원도, 온라인), dtype: float64
